Toronto bike share ridership in Q3 2016, i.e. July, August, September 2016.

Where's the most Toronto nightlife? 
- For each station, look at the % of all departures that happen between 12AM and 4AM

- Inspired by: http://iquantny.tumblr.com/tagged/citibike

What percetage of these trips are done by casual vs member riders?

In [1]:
import pandas as pd
from datetime import datetime

In [2]:
data = pd.read_csv("C:/Data/Projects/Toronto_Bikeshare/2016_Bike_Share_Toronto_Ridership_Q3.csv")
data.head(5)

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_name,to_station_name,user_type
0,53279,7-9-16 1:03,7-9-16 1:15,714,Dundas St E / Regent Park Blvd,Danforth Ave / Ellerbeck St,Member
1,53394,7-9-16 2:15,7-9-16 2:22,417,Riverdale Park North (Broadview Ave),Dundas St E / Regent Park Blvd,Member
2,58314,7-10-16 17:04,7-10-16 17:36,1904,Dundas St E / Regent Park Blvd,Queen St W / Close Ave,Member
3,60784,7-11-16 1:45,7-11-16 1:58,784,Union Station,Dundas St E / Regent Park Blvd,Member
4,93164,7-18-16 13:35,7-18-16 13:42,443,Front St W / Blue Jays Way,Front St / Yonge St (Hockey Hall of Fame),Member


In [3]:
# Functions to parse string date and time to just a datetime time, and to parse string time to datetime time

def parse_date(data):
    dt = datetime.strptime(data, "%m-%d-%y %H:%M")
    parsed_date = datetime.strftime(dt, "%H:%M")
    return parsed_date

def parse_time(data):
    dt = datetime.strptime(data, "%H:%M")
    parsed_time = datetime.strftime(dt, "%H:%M")
    return parsed_time

In [4]:
data["departure_time"] = data["trip_start_time"].apply(parse_date)
data.head()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_name,to_station_name,user_type,departure_time
0,53279,7-9-16 1:03,7-9-16 1:15,714,Dundas St E / Regent Park Blvd,Danforth Ave / Ellerbeck St,Member,01:03
1,53394,7-9-16 2:15,7-9-16 2:22,417,Riverdale Park North (Broadview Ave),Dundas St E / Regent Park Blvd,Member,02:15
2,58314,7-10-16 17:04,7-10-16 17:36,1904,Dundas St E / Regent Park Blvd,Queen St W / Close Ave,Member,17:04
3,60784,7-11-16 1:45,7-11-16 1:58,784,Union Station,Dundas St E / Regent Park Blvd,Member,01:45
4,93164,7-18-16 13:35,7-18-16 13:42,443,Front St W / Blue Jays Way,Front St / Yonge St (Hockey Hall of Fame),Member,13:35


In [5]:
data["from_station_name"].value_counts().head()

Bay St / Queens Quay W (Ferry Terminal)    6563
York St / Queens Quay W                    6409
Widmer St / Adelaide St                    5710
Union Station                              5668
Simcoe St / Wellington St W                5646
Name: from_station_name, dtype: int64

In [6]:
def make_trip_df(data, col_name):
    # Get all the stations
    stations = data["from_station_name"].value_counts().index.tolist()
    # Get number of trips
    values = data["from_station_name"].value_counts().tolist()
    # Create dataframe of just stations and their total number of trips
    # Set index to the station name
    trip_df = pd.DataFrame({"station": stations, col_name: values})
    trip_df.set_index("station", inplace = True)
    return trip_df

In [7]:
total_trips = make_trip_df(data, "total_departures")
total_trips.head(5)

,total_departures
station,
Bay St / Queens Quay W (Ferry Terminal),6563
York St / Queens Quay W,6409
Widmer St / Adelaide St,5710
Union Station,5668
Simcoe St / Wellington St W,5646


In [8]:
start_time = parse_time("0:00")
end_time = parse_time("4:00")
# Get trips that occured between 12 and 4
late_night = data[(data["departure_time"] >= start_time) & (data["departure_time"] <= end_time)]
late_trips = make_trip_df(late_night, "late_departures")
late_trips.head(5)

,late_departures
station,
Bay St / Queens Quay W (Ferry Terminal),985
York St / Queens Quay W,982
Dundas St / Yonge St,954
Widmer St / Adelaide St,900
Queen St W / Portland St,776


In [9]:
# Look at member only trips
members = data[data["user_type"] == "Member"]
# Get trips that occured between 12 and 4
late_member = members[(members["departure_time"] >= start_time) & (members["departure_time"] <= end_time)]
late_member_trips = make_trip_df(late_member, "late_member_departures")
late_member_trips.head(5)

,late_member_departures
station,
Queen St W / Portland St,581
Widmer St / Adelaide St,569
Dundas St / Yonge St,566
Bay St / College St (East Side),512
Simcoe St / Queen St W,479


In [10]:
# Join the 3 dataframes on station key
late_trip_df = late_trips.join(total_trips)
late_trip_df = late_trip_df.join(late_member_trips)
# Get percent of late night trips
late_trip_df["total_percent"] = round((late_trip_df["late_departures"]/late_trip_df["total_departures"])*100)
late_trip_df["member_percent"] = round((late_trip_df["late_member_departures"]/late_trip_df["late_departures"])*100)
# Sort descending by total percent of late trips
late_trip_df.sort_values("total_percent", ascending = False, inplace = True)
late_trip_df.head(10)

,late_departures,total_departures,late_member_departures,total_percent,member_percent
station,,,,,
Exhibition GO (Atlantic Ave),201,627,114,32.0,57.0
Strachan Ave / Princes' Blvd,361,1234,99,29.0,27.0
80 Clinton St (North of College),179,698,119,26.0,66.0
Central Tech (Harbord St),128,501,101,26.0,79.0
Ontario Place Blvd / Remembrance Dr,575,2320,176,25.0,31.0
Ossington Ave / Harrison St,121,529,50,23.0,41.0
Dufferin St / Bloor St,100,452,56,22.0,56.0
Margueretta St / College St W,67,300,28,22.0,42.0
Church St / Wood St,618,2917,188,21.0,30.0


So the most popular late night spots are the Exhibition GO station (Liberty Village), Strachan Ave/Princes' Boulevard (the watefront), Little Italy, Bloor and Bathurst, and Ontario Place, where there seems to be a lot of parties. Interestingly, waterfront and Ontario place have the lowest number of member ridership at this time, and a higher percentage of casual riders. People must be ok with drunkenly hopping on a bike at these locations. On the other hand, in Little Italy and south of Bloor street, there must be more member riders who plan to bike home that night, and possibly non members who just walk home from their night out.